# Descriptive statistics
*Gries, Chapter 3, pp. 115-135*

In this sprint, we'll cover some of the most basic "models" in statistics, i.e. one-number summaries of univariate data series. By **univariate**, we mean that we are restricting our analysis to a *single* variable or a single group of observations (typically a single "column" in one of our datasets). We'll attempt to capture, summarize or **describe** the behaviour of this variable in a sample using a single number, which is why these measure are often called **descriptive statistics**. You might also hear the terms **aggregate statistics** or **summary statistics** for such measures, because we are *aggreggating* many numbers into a single number.

## Measure of central tendency

In [1]:
import pandas as pd

We'll start our discussion with a number of well-known **measures of central tendency**: the mode, mean and median. These try to tell you something about what values are most commonly observed in the data, but they do so in their own unique way.

Let us load the "chatdata" file, that contains information on social media text that was "donated" to a sociolinguistic research project here in Antwerp.

In [ ]:
df = pd.read_csv("../../datasets/chat/chat.tsv", sep="\t")
df

#### Mode

The mode is a useful statistic, especially for categorical variables, because it tells you which value is most common in the series. Take the `education` column, for instance, in our data, which indicates which kind of education a donator had received: which level of education was most common?

Here we will use a few new pandas methods. Remember that you can easily browse the methods for `DataFrame` and `Series` objects in the online documentation.

A great method to start with is `describe()` which will vary its output depending on the series' data type. Here, for categorical data, it gives us some count statistics. The most common categorical value (`top` in the output) is the mode.

In [ ]:
df.education.describe()

We could also see this by the individual value counts...

In [ ]:
df.education.value_counts()

... which can also be normalized, if needed:

In [ ]:
df.education.value_counts(normalize=True)

And finally, we can access the mode directly with a method on the series object:

In [ ]:
df.education.mode()

The main drawback of the mode, as an aggregate statistic, is that it doesn't tell how representative this number for the series as a whole; it only tells you which value is most common overall.

#### Mean

An even more commonly used measure of central tendency is the mean, or the average value in your series. You can obtain this measure by dividing all values in the series by the sum of the entire series. Let us calculate, for instance, the mean number of emoticons in the donated text (these are cumulative counts for each user):

In [ ]:
df.emoticons.mean()

From high school, you'll remember how we can calculate this manually:

In [ ]:
df.emoticons.sum() / df.emoticons.size

This is the so-called **geometric** mean. (There's also "harmonic mean" discussed in the handbook, but that is so infrequently used that we'll skip it here.) The mean has the nice property that is "a best guess" for a random variable: whenever you have to guess what a random new value might be, the mean will always be your best guess.

#### Median

The median is often mentioned in the same context as the mean, although their behaviour is rather different. The median is the middle value of the values in your series after ranking them (whether the order is ascending or descending doesn't matter). If your data has an even number of values, there won't be a single value in the middle, so you'll have to take the (geometric) mean of the *two* middle variables.

In [ ]:
df.emoticons.median()

Wow, that's big difference in comparison to the mean! This has to do with two issues: first all, the number of emoticons listed in this columns is an "absolute" count; the variable hasn't been normalized in the light of the number of word tokens that were actually submitted by a user (this count is available from the `nr_tokens` column).

Remember that `describe()` will give us all sorts of useful information all at once. (Note: The **median** is also called the 50th percentile value, and is one of the standard **quantiles** we will discuss below.)

In [ ]:
df.emoticons.describe()

##### Exercise
> As an exercise, create a new vector that contains the relative emoticon frequency in the user's text. Calculate the mean and median for this new vector.

In [ ]:
relative_emoticons = df.emoticons / df.nr_tokens
relative_emoticons

In [ ]:
relative_emoticons.describe()

As you can see, normalizing our data didn't change the situation much: there's still a huge difference between the mean and median for the sample. This has to do with the fact that most users don't use emoticons all that much, but some of them have an extremely high proportion of emoticons in their texts. Data points with extreme values are often called **outliers** in statistics. Later on, we'll see some strategies that you can apply to automatically detect (and isolate or remove) these.

The mean is easily affected by outliers, but the median isn't. We can easily demonstrate that with the following. Can you explain what is going on here?

In [ ]:
# Here is a new trick with f-strings! Formatting can be added after the interpolated variable inside
# the {}. This one (.3f) formats the numbers as a float rounded to three decimal places.

print(f"Emoticons mean: {df.emoticons.mean():.3f}; Median: {df.emoticons.median():.3f}")

# A pandas trap!! If we do not copy() the series, we receive a "reference" to the series in the
# original dataframe, and the assignment will change *both* values -- try removing the .copy() method
# and see what happens...

distorted = df.emoticons.copy()
distorted[0] = 100000
print(f"Distorted mean: {distorted.mean():.3f}; Median: {distorted.median():.3f}")

Summing up, the median has two clear advantages over the mean:
- it's much less sensitive to outliers
- it's a number that actually occured in your data: note that families cannot have "1.2 children"
- it can be more interesting for skewed, non-normal distributions (*more on that later*)

<img src="https://pbs.twimg.com/media/EDANCjJXkAAOSjO?format=jpg&name=900x900"/>

## Measures of dispersion

#### Range

More immediately useful is the **range** of a (numeric) variable. If you have calculated a measure of central tendency, like the mean, that still doesn't tell you what the **spread** is of that variable around the mean (both to the left and to the right). One useful, yet pretty native way to get a sense of this spread, is just to report the *difference between the maximum and minimum value that was observed in the sample*; this is exactly was is meant by the "range". For the number of donated tokens, for instance, the range already suggests a pretty large difference between the most generous and least generous donator:

In [ ]:
print(f"min: {df.nr_tokens.min()} max: {df.nr_tokens.max()}")
print(f"range: {df.nr_tokens.max() - df.nr_tokens.min()}")

#### Quantiles and quartiles

A very useful way to characterize the spread around some sort of central tendency are **quantiles**, and **quartiles**, which are specific kind of quantile. To compute the quantiles for a variable, we first order all the values observed for that variable (in ascending order). Going through the sorted values, a quantile then describes by which value you have seen some proportion or percentage of all your data. This is shown by default in the `describe()` method for numeric values.

By default, this function returns the quantiles for the following percentages: 0, 25, 50, 75, and 100. These are indeed the most common thresholds that are used to report quantiles. The quantiles for the values 25, 50 and 70, even take the special name of **quartiles**, because they will evenly divide the values in four quarters.

If we look at the output, we can again see some interesting trends: the first 25% percent of the donated texts were only 74 tokens long or shorter. The range between the 100% and 75% quartile is much wider. Basically, the length range of text donations in the lower quarter of our ranked data is much smaller than that in the upper quarter of our data. (Looking at the 50% quartile, that also make sense, since token counts cannot be negative...)

A dispersion measure that is commonly reported is the **IQR** or *interquartile-range*, or the difference (AKA "range") between the 75% quartile and the 25% quartile. Unlike the "vanilla" range, the the IQR will be less affected by extreme outliers.

In [ ]:
df.nr_tokens.describe()

We can also get any quantile we like with the `quantile()` method...

In [ ]:
df.nr_tokens.quantile(0.72)

> EXERCISE
>
> So why don't you write a way to calculate the IQR as above?

In [17]:
# answer here

##### Question
> Will the 50% quantile typically be closer to the mean for a variable or to the median?

Quartiles might seem a little abstract now, but they play an important role in a plotting family that is known as the "boxplot", which  we'll cover in the next sprint.

#### Standard deviation (and variance)

Remember, in this section, we are looking into ways of capturing the dispersion of a variable, how far the values generally are from a measure of central tendency, like the mean. One obvious and intuitive way to do this, would be to just look at the average dispersal of values, i.e. the average **absolute** difference between each value from the mean of the sample. That measure is easy enough to calculate, after what we've seen before:

In [ ]:
absolute_dev = (df.emoticons - df.emoticons.mean()).abs()
absolute_dev.mean()

Unfortunately, this measure has fallen out of grace in recent years. In fact, the undisputed queen of dispersal measures (at least for numeric data) is the **standard deviation**. Broadly speaking, this measure also captures how strongly values deviate from the mean overall: the smaller the standard deviation, the better a summary the mean is. The larger the standard deviation, the less useful the mean, since we know that many values are far from it. Intuitively speaking, that should make sense to you, but mathematically, the calculation of the standard deviation is a bit more convoluted.

Mathematically, the standard devation is defined as the square root of another dispersion measure that might encounter, the **variance**. It's easy to confuse the two, so be careful. The variance, which we need to calculate first, is defined as follows, for a variable $x$ with, $n$ observations, and a mean $\bar{x}$: 

\begin{equation}
var = \frac{\sum_{i=1}^{n} (x_i - \bar{x})^{2}}{(n - 1)}
\end{equation}

##### Exercise
> Can you unravel the math in this formula and manually calculate the variance for the `nr_tokens` column in our data? Check the integrity of your calculation using the dedicated `var()` function. (Hint: it's a pretty big value!)

In [19]:
# answer here

The standard deviation is the square root of the variance, so you can obtain it as follows:

In [ ]:
# A lazy trick. To use the sqrt() method we would need to "import math", but we can raise to the
# power of 1/2 without importing anything. :)
var ** (1 / 2)

It's equivalent to the output of the dedicated series method `.std()`:

In [ ]:
df.nr_tokens.std() == var ** (1 / 2)

The standard deviation is a ubiquitous measure in statistics, so be prepared to see a lot of it. In fact, whenever you report a mean, it's **good practice** to also report the standard deviation (or some other measure of dispersal). As mentioned above, the standard deviation gives you an idea of what the actual worth of the mean is that you report. The mean and standard deviation are also important in the context of the normal distribution, which we'll cover in-depth in an upcoming sprint.

You can also apply the `describe()` method we have been using to a whole data frame, and it will summarize all (numeric) columns, by default, or optionally every column (look up the documentation for this!)

In [ ]:
df.describe(include="all")

## Centering and standardization (z-scores)

**Normalizing** your data is crucial in all quantitative approaches to data analysis, in machine learning as well as statistics. By normalization, we mean that we project the range original values in a sample onto a more controlled scale (such as [$0 \leftrightarrow 1$] or [$-1 \leftrightarrow 1$]). That makes it easier to compare variables directly, especially if those original ranges are on entirely different scales or ranges. In fact, whenever you work with any data, you need a very good reason *not* to normalize your data, since it comes with many benefits (and only a single disadvantage).

One very common way to normalize your variable is to make use of **$z$-scores**. To convert a vector of values into their **$z$-scores**, two operations are required, that each come with their own benefits: **centering** and **scaling** (also called **standardization**). Centering is the easiest operation of the two: you just need to subtract the mean value from every value:

In [25]:
centered = df.nr_tokens - df.nr_tokens.mean()

This operation has two really cool effects. First of all, the mean of your centered data will be identical to zero:

In [ ]:
centered.mean()

(Or at least: it will be very close... 😊)

This has another, even better effect: the **sign** of a centered value (whether it's positive or negative) will immediately tell whether the value was lower, or higher than the sample's mean. 

To get the actual z-scores, one further step is needed: we will divide the centered values by the standard deviation (of the original values):

In [ ]:
centered / df.nr_tokens.std()

In doing so, what we hope to hope to capture is how far each value each is away from the mean and we express that distance, not as an actual difference, but as the "number of standard deviations" that the value is "off".

Pandas does many things, but not everything. Here we will bring in another indispensible tool for data science in Python -- `scikit-learn` (often just called `sklearn`)

In [28]:
from sklearn.preprocessing import StandardScaler

This looks a little complicated, but the reason is that almost everything in the `sklearn` package follows a very standard API with consistent method names like `.fit()` `.transform()`, and the one-step combination we will use here `.fit_transform()`. "Fitting" in this case just involves internally calculating the mean and standard deviation (as we just did manually) and "transforming" does the centering and division.

Finally, the `StandardScaler` returns its values as a Numpy `array`. We will talk a lot more about Numpy in future sessions, but for now we are going to convert the array back into a pandas `DataFrame`

In [ ]:
ss = StandardScaler()
pd.DataFrame(ss.fit_transform(df[["nr_tokens"]])).head(4)

Note that the centering and scaling can we controlled independently; there are times when we won't want to center, for example.

In [ ]:
ssnm = StandardScaler(with_mean=False)
pd.DataFrame(ssnm.fit_transform(df[["nr_tokens"]])).head(4)

In [ ]:
ssnsd = StandardScaler(with_std=False)
pd.DataFrame(ssnsd.fit_transform(df[["nr_tokens"]])).head(4)

By the way, the module `sklearn.preprocessing` has *many* scalers available for different needs, some of which we will use in the future. If you need more Python in your life, you can see some nice visualisations of different scaling effects [here](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html)

The only drawback of converting values to $z$-scores has to do with *interpretation*: whereas the original units of the variable are typically straightforward to understand (e.g. frequencies, milimeters, years, ...), the **units** in which $z$-scores are expressed no longer correspond to a measure in the outside, physical world. The unit of expression now has become the standard deviation, which is much harder to interpret intuitively.

Finally, one classic application of z-scores is to remove outliers. If a value is, let's say, 10 (!) standard deviations removed from the mean (in absolute terms), you could use that as a formal argument that the data point is an **outlier**. Removing such outliers could then proceed as follows:

In [ ]:
# Pandas has a lot of quick plots built in. Don't worry too much about this yet, because we have a
# whole session on visualisations! Here we just look at where the `nr_tokens` values *usually* lie. We
# use a log scale so the (few) outliers are easier to see...

df.nr_tokens.plot.hist(log=True)

Let's add the scaled values as a new column so we can see everything at once...


In [ ]:
ss = StandardScaler()
df["nr_tokens_scaled"] = ss.fit_transform(df[["nr_tokens"]])
df

And now we'll use our subsetting tricks again to extract all of the non-outliers (according to the $z$-level we defined), and then the opposite, to look at the outliers...

In [ ]:
# A new trick! There are many "vectorized" methods for pandas objects that apply e.g. an operation
# to an entire Series -- here we use the `abs()`` method.

df[df.nr_tokens_scaled.abs() < 10]

In [ ]:
df[df.nr_tokens_scaled.abs() >= 10]

Look at Subject 571 go!! 🥇​🥇😂​🤣​🤣​🤣​😂​😂​😍​🥰

As you can see, we effectively removed four outliers from the normalized data in this way. Removing outliers is not always a good idea -- when is a value extreme enough to be ignored? -- but $z$-scores at least give you a **principled** way to do so.

```
Version History

Current: v1.0.2

9/11/24: 1.0.0: first draft, BN
9/11/24: 1.0.1: fixed typos and proofread, MK
13/10/24: 1.0.2: move to public, BN
```